In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from imblearn.over_sampling import SMOTE
from IPython.display import display
from sklearn.ensemble import RandomForestRegressor

In [2]:
pop = pd.read_csv(r'C:\Users\User\Desktop\EDurHack\Dataset\cleanData\woodchucks_with_wood_volume.csv')

pop = pop.fillna(0)

years = pop['year'].unique().tolist()

lats = pop['latitude'].unique().tolist()

display(pop)
print(years)
print(lats)

,year,latitude,longitude,estimated_woodchuck_population,VOLCF_AC_UNADJ,wood_chucked_per_woodchuck_lbs,total_wood_chucked_lbs
0,2018,39.9,-75.6,3441.084157,17.493488,0.000000,0.000000
1,2018,39.9,-75.4,9445.985464,17.493488,0.000000,0.000000
2,2018,39.9,-75.3,7222.017160,17.493488,0.000000,0.000000
3,2018,39.9,-75.2,11869.568244,17.493488,0.000000,0.000000
4,2018,40.0,-80.0,295.736902,97.186046,1.001197,296.090767
...,...,...,...,...,...,...,...
899,2024,41.9,-77.8,50.000000,3176.039981,39.681571,1984.078533
900,2024,42.0,-80.2,292.625815,371.250695,4.444336,1300.527421
901,2024,42.0,-79.6,50.000000,371.250695,4.444336,222.216796
902,2024,42.1,-80.1,6105.471860,371.250695,4.444336,27134.767856


[2018, 2019, 2020, 2021, 2022, 2023, 2024]
[39.9, 40.0, 40.1, 40.2, 40.3, 40.4, 40.5, 40.6, 40.7, 40.8, 41.0, 41.1, 41.2, 41.5, 41.8, 41.9, 42.2, 39.7, 39.8, 40.9, 41.3, 41.6, 41.4, 41.7, 42.0, 42.1]


In [6]:
def forecast(years, lats):
    all_results = []  # collect results across all years, lats, and lons

    for year in years:
        for lat in lats:
            lons = pop[pop['latitude'] == lat]['longitude'].unique().tolist()
            
            for lon in lons:
                location_data = pop[
                    (pop['latitude'] == lat) &
                    (pop['longitude'] == lon)
                ].copy()
                
                location_data = location_data.sort_values('year').reset_index(drop=True)

                location_data['time_index'] = range(len(location_data))
                location_data['rolling_mean_3'] = (
                    location_data['total_wood_chucked_lbs']
                    .rolling(window=3, min_periods=1)
                    .mean()
                )

                if len(location_data) < 3:
                    last_year = location_data['year'].iloc[-1]
                    last_value = location_data['total_wood_chucked_lbs'].iloc[-1]

                    future = pd.DataFrame({
                        'latitude': lat,
                        'longitude': lon,
                        'year': [last_year + i + 1 for i in range(16)],
                        'total_wood_chucked_lbs': [last_value] * 16,
                        'type': 'forecast'
                    })

                else:
                    # Prepare training data (no lag features)
                    X = location_data[['year', 'time_index', 'rolling_mean_3']]
                    y = location_data['total_wood_chucked_lbs']
                
                model = RandomForestRegressor(
                    n_estimators=100,
                    max_depth=10,
                    min_samples_split=2,
                    random_state=42,
                    n_jobs=-1
                )
                model.fit(X, y)

                last_year = location_data['year'].iloc[-1]
                last_time_index = location_data['time_index'].iloc[-1]
                
                future_years = [last_year + i + 1 for i in range(16)]
                future_time_index = [last_time_index + i + 1 for i in range(15)]
                future_rolling = [location_data['rolling_mean_3'].iloc[-1]] * 15
                future_X = pd.DataFrame({
                        'year': future_years,
                        'time_index': future_time_index,
                        'rolling_mean_3': future_rolling
                    })
                
                preds = model.predict(future_X)
                future = pd.DataFrame({
                    'latitude': lat,
                    'longitude': lon,
                    'year': future_years,
                    'total_wood_chucked_lbs': preds,
                    'type': 'forecast'
                })

                # Combine historical and future
                past = location_data[['latitude', 'longitude', 'year', 'total_wood_chucked_lbs']].copy()
                past['type'] = 'historical'
                combined = pd.concat([past, future])
                all_results.append(combined)

    final_df = pd.concat(all_results).sort_values(['year', 'latitude', 'longitude']).reset_index(drop=True)
    final_df.to_csv("woodchuck_forecast_all.csv", index=False)

    return final_df

forecast(years, lats)


ValueError: All arrays must be of the same length